# Calibrating R0 Files to R1

This notebook outlines the process of calibrating R0 files to R1 files, using the TargetCalib library. If you only had access to the R0 files, and not the \_ped.tcal file (Pedestal file), R1 (calibrated waveforms) or DL1 (reduced files), then this is the point you would have to start at.

For this tutorial you need TargetDriver, TargetIO and TargetCalib installed.

## Setup

Prepare your machine and environment by following the instructions at: https://forge.in2p3.fr/projects/gct/wiki/Installing_CHEC_Software

If you do not wish to install the TARGET libraries as you will only be reading DL1 files, you can skip this tutorial.

To set up TC_CONFIG_PATH for the Transfer Functions, download svn.in2p3.fr/cta/Sandbox/justuszorn/CHECDevelopment/CHECS/Operation to a directory, and `export TC_CONFIG_PATH=...`.

Check the installation was successful by running these lines:

In [ ]:
import target_driver
import target_io
import target_calib
from os import environ
assert 'TC_CONFIG_PATH' in environ

## Files

To run this tutorial you must download a reference dataset (using the username and password Rich has sent around in emails/Slack). This dataset consists of a raw (R0) data files for a pedestal run and a ~50 p.e. illumination run.

In [ ]:
username = '***'
pw = '***'
ped_r0_url = 'https://www.mpi-hd.mpg.de/personalhomes/white/checs/data/d0000_ReferenceData/Run17438_r0.tio'
ped_url = 'https://www.mpi-hd.mpg.de/personalhomes/white/checs/data/d0000_ReferenceData/Run17438_ped.tcal'
r0_url = 'https://www.mpi-hd.mpg.de/personalhomes/white/checs/data/d0000_ReferenceData/Run17473_r0.tio'

In [ ]:
!mkdir refdata
#!wget --user $username --password $pw -P refdata $ped_r0_url  # LARGE FILE - only download for generating pedestal file
!wget --user $username --password $pw -P refdata $ped_url
!wget --user $username --password $pw -P refdata $r0_url

In [ ]:
ped_r0_path = "refdata/Run17438_r0.tio"
ped_path = "refdata/Run17438_ped.tcal"
r0_path = "refdata/Run17473_r0.tio"

## Pedestal Generation

This executable produces the pedestal file (_ped.tcal) from an R0 run.

In [ ]:
!generate_ped -h

In [ ]:
!generate_ped -i $ped_r0_path

## Applying Calibration

This executable applies the calibration files (pedestal and transfer function) to an R0 file to produce a R1 file containing calibrated waveforms. The "-x" argument instructs the executable to find the correct TFs for each module by looking at the module serial numbers in the header of the R0 file, and find the appropriate TF files inside $TC_CONFIG_PATH

In [ ]:
!apply_calibration -h

In [ ]:
!apply_calibration -i $r0_path -p $ped_path -x

## Next Step

You now have an "_r1.tio" file containing the R1-calibrated waveforms. These waveforms have all the calibration required for data read out from a TARGET module, and can now be analysed with common signal processing techniques. The next step in the CHECLabPy pipeline is to reduce the waveforms using the extract_dl1.py script, detailed in the "2_Reducing_R1_to_DL1.ipynb" tutorial.